# Mammary Adenoma vs Adenocarcinoma 추출 PoC (v1)
이 노트북은 앞서 설명한 단계별 필터링 절차를 적용하여 `Data/조직검사 결과 매칭(2024)_utf8_pruned.csv`에서 **Mammary Adenoma**와 **Mammary Adenocarcinoma**를 구분합니다.
<sub>규칙 기반 1차 필터와 수동 검토 루프를 묶어 약한 감독(weak supervision) 방식으로 레이블링 비용을 낮추는 PoC 흐름을 검증합니다.</sub>

In [1]:
import pandas as pd
from pathlib import Path
import pyarrow  # ensure parquet engine is available

pd.options.display.max_colwidth = 120

DATA_PATH = Path("../../Data/조직검사 결과 매칭(2024)_utf8_pruned.parquet")
assert DATA_PATH.exists(), f"Missing data file: {DATA_PATH}"

raw_df = pd.read_parquet(DATA_PATH, engine="pyarrow")
raw_df.head()


,INSP_RQST_NO,FOLDER,FILE_NAME,INSP_SRVC_NM,RESULT_PDF,DIAGNOSIS,GROSS_FINDINGS,COMMENTS,MICROSCOPIC_FINDINGS,SITE,SNAPSHOT
0,20240101-113-0002,S24-00001,S24-00001#1###3.svs,Histopathology (1 Site/Lesion)-국내,None,"Mast cell tumor (Well-differentiated), completely excised","Specimen: Skin<br>Appearance: 단독 single,soft,유동적 movable, Size: 0.5X0.5X0.5cm<br>Operation: excisional<br>Recurrence...","의뢰된 좌측 후지-옆구리 쪽 피부 종양은 비만세포종(mast cell tumor)으로 확인되었습니다. Mast cell tumor는 고양이 피부 종양의 약 21%를 차지하며, 분화 정도나 조직학적 분류와는 관...","의뢰된 조직의 장축면을 제작-판독하였습니다. 경계가 비교적 명확한 직경 약 4.3 mm의 mass는 원형 세포로 구성되어 있었으며, 종양세포는 잘 분화된 비만세포로 확인되었습니다. 종양 세포는 metachro...",site1,https://ipeak.greenvet.co.kr/siteImage/17143/1-1.jpg
1,20240101-113-0002,S24-00001,S24-00001#1###3.svs,Histopathology (1 Site/Lesion)-국내,None,"Mast cell tumor (Well-differentiated), completely excised","Specimen: Skin<br>Appearance: 단독 single,soft,유동적 movable, Size: 0.5X0.5X0.5cm<br>Operation: excisional<br>Recurrence...","의뢰된 좌측 후지-옆구리 쪽 피부 종양은 비만세포종(mast cell tumor)으로 확인되었습니다. Mast cell tumor는 고양이 피부 종양의 약 21%를 차지하며, 분화 정도나 조직학적 분류와는 관...","의뢰된 조직의 장축면을 제작-판독하였습니다. 경계가 비교적 명확한 직경 약 4.3 mm의 mass는 원형 세포로 구성되어 있었으며, 종양세포는 잘 분화된 비만세포로 확인되었습니다. 종양 세포는 metachro...",site1,https://ipeak.greenvet.co.kr/siteImage/17143/1-2.jpg
2,20240101-121-0003,S24-00003,S24-00003#1###8.svs,Histopathology (1 Site/Lesion)-국내,None,"Apocrine or mammary adenoma, completely excised, see Comments","Specimen: Other(생식기 주변 피하)<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph:...","의뢰된 생식기 주위 피하 종괴는 아포크린샘 유래의 양성 종양으로 진단되었습니다. 피부에 분포하는 땀샘과 유선, 항문낭샘 등이 모두 아포크린샘에 속하며, 해부학적인 위치와 주위 정상 조직과의 연관성을 토대로 정...","샘 유래의 상피세포로 이루어진 한 개의 종양성 종괴입니다. 종양세포는 대부분 잘 분화되어 균일한 크기와 형태를 보이는 입방형 상피세포입니다. 세포 두 층 두께의 세관 구조를 형성하며 증식하였고, 일부 세관은 ...",site1,https://ipeak.greenvet.co.kr/siteImage/17145/0003_01.JPG
3,20240101-121-0003,S24-00003,S24-00003#1###8.svs,Histopathology (1 Site/Lesion)-국내,None,"Apocrine or mammary adenoma, completely excised, see Comments","Specimen: Other(생식기 주변 피하)<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph:...","의뢰된 생식기 주위 피하 종괴는 아포크린샘 유래의 양성 종양으로 진단되었습니다. 피부에 분포하는 땀샘과 유선, 항문낭샘 등이 모두 아포크린샘에 속하며, 해부학적인 위치와 주위 정상 조직과의 연관성을 토대로 정...","샘 유래의 상피세포로 이루어진 한 개의 종양성 종괴입니다. 종양세포는 대부분 잘 분화되어 균일한 크기와 형태를 보이는 입방형 상피세포입니다. 세포 두 층 두께의 세관 구조를 형성하며 증식하였고, 일부 세관은 ...",site1,https://ipeak.greenvet.co.kr/siteImage/17145/0003_02.JPG
4,20240101-102-0001,S24-00018,S24-00018#1###2.svs,Histopathology (1 Site/Lesion)-국내,None,"Mammary duct ectasia, mastitis","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.5X0.5X0.3cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...","Mammary duct ectasia는 도관이 확장된 병변으로, 주로 중성화하지 않은 암컷에서 발정주기에 따라 성호르몬의 영향으로 발생하며, 유선 종양 주위에 형성되기도 합니다.<br>의뢰된 유선 검체에서는 ...","의뢰된 유선 검체 저배율 시야에서는 다수의 유선 도관 확장이 특징적인 소견으로 확인됩니다. 병변의 경계는 적절히 확보되었습니다. 확장된 유선 사이로는 mild하게 과증식한 유선 조직이 분포하고 있으며, 기질로...",site1,https://ipeak.greenvet.co.kr/siteImage/17160/18_01.JPG


## 1. 스키마와 초기 건수 확인

<sub>스키마와 총 건수를 미리 확인해야 이후 필터링에서 열 이름 오타나 누락으로 인한 정보 손실을 방지할 수 있습니다.</sub>

In [2]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48692 entries, 0 to 48691
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   INSP_RQST_NO          48692 non-null  object
 1   FOLDER                48692 non-null  object
 2   FILE_NAME             48692 non-null  object
 3   INSP_SRVC_NM          48692 non-null  object
 4   RESULT_PDF            12 non-null     object
 5   DIAGNOSIS             48692 non-null  object
 6   GROSS_FINDINGS        48690 non-null  object
 7   COMMENTS              48692 non-null  object
 8   MICROSCOPIC_FINDINGS  48692 non-null  object
 9   SITE                  48692 non-null  object
 10  SNAPSHOT              48690 non-null  object
dtypes: object(11)
memory usage: 4.1+ MB


## 2. 텍스트 정규화 도우미
소문자 변환, HTML `<br>` 제거, 공백 정리, 그리고 느슨한 매칭을 위한 알파벳 전용 보조 문자열을 생성합니다.
<sub>서술문이 길거나 HTML 줄바꿈이 섞인 경우도 일관된 형태로 맞추기 위해 정규화를 먼저 수행합니다 (예: `Mammary adenocarcinoma<br>complete excision` → `mammary adenocarcinoma complete excision`).</sub>

In [3]:
import re

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = s.replace('<br>', ' ')
    s = re.sub(r"[^a-z0-9\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def alpha_only(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

norm_cols = ['DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS']
norm_df = raw_df.copy()
for col in norm_cols:
    norm_df[f"{col}_norm"] = norm_df[col].apply(normalize_text)
    norm_df[f"{col}_alpha"] = norm_df[col].apply(alpha_only)

norm_df[[c for c in norm_df.columns if c.endswith('_norm')]].head(10)

,DIAGNOSIS_norm,GROSS_FINDINGS_norm,COMMENTS_norm,MICROSCOPIC_FINDINGS_norm
0,mast cell tumor well differentiated completely excised,specimen skin appearance single soft movable size 0 5x0 5x0 5cm operation excisional recurrence n lymph n,mast cell tumor mast cell tumor 21 80 cutaneous mast cell tumor grading system mitotic index 5 400 10 5 mast cell tu...,4 3 mm mass metachromic granule 3 10 hpf 1 39 mm
1,mast cell tumor well differentiated completely excised,specimen skin appearance single soft movable size 0 5x0 5x0 5cm operation excisional recurrence n lymph n,mast cell tumor mast cell tumor 21 80 cutaneous mast cell tumor grading system mitotic index 5 400 10 5 mast cell tu...,4 3 mm mass metachromic granule 3 10 hpf 1 39 mm
2,apocrine or mammary adenoma completely excised see comments,specimen other appearance single size 1x1x1cm operation excisional recurrence n lymph n,,
3,apocrine or mammary adenoma completely excised see comments,specimen other appearance single size 1x1x1cm operation excisional recurrence n lymph n,,
4,mammary duct ectasia mastitis,specimen mammary appearance multiple size 0 5x0 5x0 3cm operation excisional recurrence n lymph n,mammary duct ectasia 80 mass,mild flat cubiodal
5,mammary duct ectasia mastitis,specimen mammary appearance multiple size 0 5x0 5x0 3cm operation excisional recurrence n lymph n,mammary duct ectasia 80 mass,mild flat cubiodal
6,mammary duct ectasia mastitis,specimen mammary appearance multiple size 0 5x0 5x0 3cm operation excisional recurrence n lymph n,mammary duct ectasia 80 mass,mild flat cubiodal
7,mammary duct ectasia mastitis,specimen mammary appearance multiple size 0 5x0 5x0 3cm operation excisional recurrence n lymph n,mammary duct ectasia 80 mass,mild flat cubiodal
8,fibroma,specimen skin appearance single size 0 1x0 1x0 1cm operation excisional recurrence n lymph n,,
9,fibroma,specimen skin appearance single size 0 1x0 1x0 1cm operation excisional recurrence n lymph n,,


## 3. 키워드 규칙
Adenoma와 Adenocarcinoma 각각에 대해 확정 매칭용 기본 패턴과 다른 컬럼의 보조 컨텍스트 용어를 정의합니다.
<sub>자유 서술식으로 입력된 진단명을 포착하기 위해 강한 패턴과 약한 보조 컨텍스트를 분리해 관리하고, 두 진단군을 분리하는 데 핵심이 되는 malignant 표현(`carcinoma`, `malignant`, `invasive` 등)을 별도로 모읍니다.</sub>

In [4]:
primary_patterns = {
    'adenoma': [
        r"mammary adenom",
        r"mamary adenom",  # common misspelling
        r"mammary benign adenom",
        r"apocrine or mammary adenom",
        r"mammary adenom[ae]",
        r"mammary\s+gland\s+adenom",
    ],
    'adenocarcinoma': [
        r"mammary adenocarcin",
        r"mammary carcinoma",
        r"mammary\s+gland\s+adenocarcin",
        r"mammary\s+gland\s+carcinoma",
        r"mammary malignant tumor",
    ],
}

# 전문가가 확실한 표현을 추가할 때 사용 (예: r"mammary gland adenoma")
custom_primary_patterns = {
    'adenoma': [],
    'adenocarcinoma': [],
}

# 지원 컬럼별 기본 보조 키워드
support_keywords = {
    'adenoma': {
        'COMMENTS': ["adenoma", "유선", "apocrine", "benign", "adenomatous"],
        'MICROSCOPIC_FINDINGS': ["adenoma", "lobule", "capsulated", "ductal", "benign"],
        'GROSS_FINDINGS': ["mammary", "乳腺", "mamary", "mass", "nodule"],
    },
    'adenocarcinoma': {
        'COMMENTS': ["adenocarcinoma", "carcinoma", "malignant", "invasive", "metastatic", "anaplastic"],
        'MICROSCOPIC_FINDINGS': ["carcinoma", "adenocarcinoma", "malignant", "invasive", "anaplastic", "pleomorphic"],
        'GROSS_FINDINGS': ["mammary", "mamary", "ulcer", "hemorrhage", "necrosis", "irregular"],
    },
}

# 전문가가 보조 키워드를 추가할 때 사용 (컬럼별 리스트를 확장)
custom_support_keywords = {
    'adenoma': {
        'COMMENTS': [],
        'MICROSCOPIC_FINDINGS': [],
        'GROSS_FINDINGS': [],
    },
    'adenocarcinoma': {
        'COMMENTS': [],
        'MICROSCOPIC_FINDINGS': [],
        'GROSS_FINDINGS': [],
    },
}

# 느슨한 공존 판정을 위한 기본 용어
loose_terms = {
    'adenoma': ["mammary", "mamary"],
    'adenocarcinoma': ["mammary", "mamary"],
}

all_primary_patterns = {
    label: pats + custom_primary_patterns.get(label, [])
    for label, pats in primary_patterns.items()
}

merged_support_keywords = {
    label: {
        col: keywords + custom_support_keywords.get(label, {}).get(col, [])
        for col, keywords in support_map.items()
    }
    for label, support_map in support_keywords.items()
}


## 4. 점수 산정 함수
- 진단 컬럼에 확정 패턴 매칭 시 +2 (adenoma/adenocarcinoma 별도 집계)
- 알파벳 전용 문자열에서 'mammary/mamary'와 핵심 진단 토큰(adenoma vs carcinoma)이 함께 등장하면 +1
- 지원 컬럼에 보조 키워드가 있으면 컬럼당 +1
- 최종 라벨: 점수가 더 큰 쪽을 우선하며, 둘 다 0이면 `none`, 동률이면 `uncertain`
<sub>양성과 악성을 동일한 `mammary` 해부 부위 안에서 구분하기 위해 두 집합을 병렬로 점수화합니다. 동시에 등장하거나 애매한 경우를 `uncertain`으로 남겨 수동 검토 대상을 명확히 합니다.</sub>

In [5]:

import pandas as pd

mammary_context_terms = ['mammary', 'mamary']
mammary_context_korean = ['유선', '乳腺']

def has_mammary_context(row):
    raw_fields = ' '.join(
        str(row.get(col, ''))
        for col in ['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS']
        if isinstance(row.get(col, ''), str)
    ).lower().replace('<br>', ' ')
    if any(term in row['DIAGNOSIS_norm'] for term in mammary_context_terms):
        return True
    return any(term in raw_fields for term in mammary_context_terms + mammary_context_korean)

def score_row(row):
    scores = {'adenoma': 0, 'adenocarcinoma': 0}
    diag_norm = row['DIAGNOSIS_norm']
    diag_alpha = row['DIAGNOSIS_alpha']
    diag_tokens = set(diag_alpha.split())

    for label in scores:
        # primary patterns
        if any(re.search(pat, diag_norm) for pat in all_primary_patterns[label]):
            scores[label] += 2

        # loose co-occurrence within diagnosis
        if label == 'adenoma':
            if diag_tokens.intersection(loose_terms[label]) and 'adenoma' in diag_tokens:
                scores[label] += 1
        else:
            if diag_tokens.intersection(loose_terms[label]) and (
                'carcinoma' in diag_tokens or 'adenocarcinoma' in diag_tokens
            ):
                scores[label] += 1

        # supporting columns
        for col, keywords in merged_support_keywords[label].items():
            text = row[f"{col}_norm"]
            if any(kw in text for kw in keywords):
                scores[label] += 1

    mammary_ok = has_mammary_context(row)

    if scores['adenoma'] == 0 and scores['adenocarcinoma'] == 0:
        label = 'none'
    elif not mammary_ok:
        label = 'uncertain'
    elif scores['adenoma'] > scores['adenocarcinoma']:
        label = 'mammary_adenoma'
    elif scores['adenocarcinoma'] > scores['adenoma']:
        label = 'mammary_adenocarcinoma'
    else:
        label = 'uncertain'

    return pd.Series({
        'score_adenoma': scores['adenoma'],
        'score_adenocarcinoma': scores['adenocarcinoma'],
        'label': label,
    })

scores_df = norm_df.apply(score_row, axis=1)
norm_df = pd.concat([norm_df, scores_df], axis=1)

norm_df[['DIAGNOSIS', 'score_adenoma', 'score_adenocarcinoma', 'label']].head(20)


,DIAGNOSIS,score_adenoma,score_adenocarcinoma,label
0,"Mast cell tumor (Well-differentiated), completely excised",0,0,none
1,"Mast cell tumor (Well-differentiated), completely excised",0,0,none
2,"Apocrine or mammary adenoma, completely excised, see Comments",3,0,mammary_adenoma
3,"Apocrine or mammary adenoma, completely excised, see Comments",3,0,mammary_adenoma
4,"Mammary duct ectasia, mastitis",1,1,uncertain
5,"Mammary duct ectasia, mastitis",1,1,uncertain
6,"Mammary duct ectasia, mastitis",1,1,uncertain
7,"Mammary duct ectasia, mastitis",1,1,uncertain
8,섬유종(Fibroma),0,0,none
9,섬유종(Fibroma),0,0,none


In [6]:
# adenoma로 분류된 행 미리보기 (전수 확인용)
adenoma_rows = norm_df[norm_df['label'] == 'mammary_adenoma']
adenoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma']]


,DIAGNOSIS,COMMENTS,MICROSCOPIC_FINDINGS,GROSS_FINDINGS,score_adenoma
2,"Apocrine or mammary adenoma, completely excised, see Comments","의뢰된 생식기 주위 피하 종괴는 아포크린샘 유래의 양성 종양으로 진단되었습니다. 피부에 분포하는 땀샘과 유선, 항문낭샘 등이 모두 아포크린샘에 속하며, 해부학적인 위치와 주위 정상 조직과의 연관성을 토대로 정...","샘 유래의 상피세포로 이루어진 한 개의 종양성 종괴입니다. 종양세포는 대부분 잘 분화되어 균일한 크기와 형태를 보이는 입방형 상피세포입니다. 세포 두 층 두께의 세관 구조를 형성하며 증식하였고, 일부 세관은 ...","Specimen: Other(생식기 주변 피하)<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph:...",3
3,"Apocrine or mammary adenoma, completely excised, see Comments","의뢰된 생식기 주위 피하 종괴는 아포크린샘 유래의 양성 종양으로 진단되었습니다. 피부에 분포하는 땀샘과 유선, 항문낭샘 등이 모두 아포크린샘에 속하며, 해부학적인 위치와 주위 정상 조직과의 연관성을 토대로 정...","샘 유래의 상피세포로 이루어진 한 개의 종양성 종괴입니다. 종양세포는 대부분 잘 분화되어 균일한 크기와 형태를 보이는 입방형 상피세포입니다. 세포 두 층 두께의 세관 구조를 형성하며 증식하였고, 일부 세관은 ...","Specimen: Other(생식기 주변 피하)<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph:...",3
56,"Mammary gland adenoma, completely excised","개의 유선 종양은 중성화하지 않은 암컷 개체에서 가장 흔한 종양입니다. 성호르몬(e.g. estrogen, progestin)이 risk factor로 작용하여 첫 번째~두 번째 발정 이전에 중성화가 이루어지...","의뢰된 유선 검체 저배율 시야에서는 피막화된 유선 종양이 확인됩니다. 종양은 도관 구조를 대체로 유지하고 있고, 일부 도관이 확장되며 cystic한 양상으로 관찰됩니다. 확장된 도관 내강으로는 유즙이 차 있으...","Specimen: Mammary<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",4
57,"Mammary gland adenoma, completely excised","개의 유선 종양은 중성화하지 않은 암컷 개체에서 가장 흔한 종양입니다. 성호르몬(e.g. estrogen, progestin)이 risk factor로 작용하여 첫 번째~두 번째 발정 이전에 중성화가 이루어지...","의뢰된 유선 검체 저배율 시야에서는 피막화된 유선 종양이 확인됩니다. 종양은 도관 구조를 대체로 유지하고 있고, 일부 도관이 확장되며 cystic한 양상으로 관찰됩니다. 확장된 도관 내강으로는 유즙이 차 있으...","Specimen: Mammary<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",4
64,"Mammary complex adenoma, completely excised","개의 유선 종양은 중성화하지 않은 암컷 개체에서 가장 흔한 종양입니다. 성호르몬(e.g. estrogen, progestin)이 risk factor로 작용하여 첫 번째~두 번째 발정 이전에 중성화가 이루어지...",의뢰된 좌측 4번 유선 검체 저배율 시야에서는 근상피(myoepithelium)의 증식이 동반된 유선 종양이 확인됩니다. 종양은 피막화되어 있으며 적절한 경계 확보가 이루어졌습니다. 종양은 도관 구조를 대체로...,"Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.8X0.6X0.4cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...",2
...,...,...,...,...,...
48678,<p>Sebaceous adenoma</p>,"<p>의뢰된 견갑부 피부 종괴는 피지샘 유래의 양성 종양인 피지샘 선종(sebaceous adenoma)으로 확인되었습니다. 이러한 피지샘 선종은 개의 피부에서 비교적 흔하게 발생하며, 단일 종괴로 발생할 수...","<p>종괴의 1단면이 제작-판독되었습니다. 진피에 분포하는 직경 약 6.6 mm의 결절은 증식된 기름샘 유래 세포(sebocytes)로 구성되어 있었으며, 전체 병변의 경계는 명확하였습니다. 증식된 피지샘 세...","<p>Specimen: Other(우측 견갑외측 피지낭종 의심)<br>Appearance: 다발 multiple,hard, Size: 1X1X1cm<br>Operation: incisional<br>Recur...",1
48679,<p>Sebaceous adenoma</p>,"<p>의뢰된 견갑부 피부 종괴는 피지샘 유래의 양성 종양인 피지샘 선종(sebaceous adenoma)으로 확인되었습니다. 이러한 피지샘 선종은 개의 피부에서 비교적 흔하게 발생하며, 단일 종괴로 발생할 수...","<p>종괴의 1단면이 제작-판독되었습니다. 진피에 분포하는 직경 약 6.6 mm의 결절은 증식된 기름샘 유래 세포(sebocytes)로 구성되어 있었으며, 전체 병변의 경계는 명확하였습니다. 증식된 피지샘 세...","<p>Specimen: Other(우측 견갑외측 피지낭종 의심)<br>Appearance: 다발 multiple,hard, Size: 1X1X1cm<br>Operation: incisional<br>Recur...",1
48687,"<p>Sebaceous adenoma, inflammed</p>","<p>SITE 2로 의뢰된 두경부 종괴는 피지샘 유래의 양성 종양인 sebaceous adenoma로 확인되었습니다. 이러한 피지샘 선종은 단독으로 발생할 수도, 피부 여러 곳에서 다발성으로도 관찰되는 경우도...",<p>의뢰 조직의 3단면이 블록과 슬라이드로 제작-판독되었습니다. 진피층에 분포하는 직경 약 4.0 mm로 측정되는 결절은 증식된 기름샘 유래 세포(sebocytes)로 구성되어 있었으며 확장된 모낭 구조물을...,"<p>Specimen: Skin<br>Appearance: 단독 single,hard, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<b...",1
48688,"<p>Sebaceous adenoma, inflammed</p>","<p>SITE 2로 의뢰된 두경부 종괴는 피지샘 유래의 양성 종양인 sebaceous adenoma로 확인되었습니다. 이러한 피지샘 선종은 단독으로 발생할 수도, 피부 여러 곳에서 다발성으로도 관찰되는 경우도...",<p>의뢰 조직의 3단면이 블록과 슬라이드로 제작-판독되었습니다. 진피층에 분포하는 직경 약 4.0 mm로 측정되는 결절은 증식된 기름샘 유래 세포(sebocytes)로 구성되어 있었으며 확장된 모낭 구조물을...,"<p>Specimen: Skin<br>Appearance: 단독 single,hard, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<b...",1


In [7]:
# adenocarcinoma로 분류된 행 미리보기 (전수 확인용)
adenocarcinoma_rows = norm_df[norm_df['label'] == 'mammary_adenocarcinoma']
adenocarcinoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenocarcinoma']]


,DIAGNOSIS,COMMENTS,MICROSCOPIC_FINDINGS,GROSS_FINDINGS,score_adenocarcinoma
12,"Mammary intraductal papillary carcinoma, Grade 1",고양이에서 발생하는 mammary gland neoplasia는 90% 이상이 악성입니다. 성호르몬이 risk factor로 작용하므로 6개월령~1년령 이전에 중성화를 통해 80~90% 정도 예방할 수 있습니...,"의뢰된 유선 검체 저배율 시야에서는 다수의 확장된 도관 내강으로 증식한 유선 종양이 확인됩니다. 종양은 도관 내강을 향해 유두상(papillary)으로 증식했으며, 기질을 향한 침습성은 낮은 편입니다. 종양의...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 3X3X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",2
13,"Mammary intraductal papillary carcinoma, Grade 1",고양이에서 발생하는 mammary gland neoplasia는 90% 이상이 악성입니다. 성호르몬이 risk factor로 작용하므로 6개월령~1년령 이전에 중성화를 통해 80~90% 정도 예방할 수 있습니...,"의뢰된 유선 검체 저배율 시야에서는 다수의 확장된 도관 내강으로 증식한 유선 종양이 확인됩니다. 종양은 도관 내강을 향해 유두상(papillary)으로 증식했으며, 기질을 향한 침습성은 낮은 편입니다. 종양의...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 3X3X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",2
14,"Mammary intraductal papillary carcinoma, Grade 1",고양이에서 발생하는 mammary gland neoplasia는 90% 이상이 악성입니다. 성호르몬이 risk factor로 작용하므로 6개월령~1년령 이전에 중성화를 통해 80~90% 정도 예방할 수 있습니...,"의뢰된 유선 검체 저배율 시야에서는 다수의 확장된 도관 내강으로 증식한 유선 종양이 확인됩니다. 종양은 도관 내강을 향해 유두상(papillary)으로 증식했으며, 기질을 향한 침습성은 낮은 편입니다. 종양의...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 3X3X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",2
15,"Mammary intraductal papillary carcinoma, Grade 1",고양이에서 발생하는 mammary gland neoplasia는 90% 이상이 악성입니다. 성호르몬이 risk factor로 작용하므로 6개월령~1년령 이전에 중성화를 통해 80~90% 정도 예방할 수 있습니...,"의뢰된 유선 검체 저배율 시야에서는 다수의 확장된 도관 내강으로 증식한 유선 종양이 확인됩니다. 종양은 도관 내강을 향해 유두상(papillary)으로 증식했으며, 기질을 향한 침습성은 낮은 편입니다. 종양의...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 3X3X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",2
48,Undifferentiated carcinoma suspect,"의뢰하신 잇몸조직에 대한 검사결과, 상피층이 증식하여 점막밑조직까지 광범위하게 침습하여 분포하고 있는 조직학적 소견을 바탕으로 암종(carcinoma)로 진단하였습니다. 또한 세포들의 형태와 배열이 비정형적인...",의뢰된 잇몸조직 3-4점이 제작-판독되었습니다. 조직은 점막상피는 없는 상태로 섬유결합조직 및 비정상적인 세포들이 결합조직 내 광범위학 분포하는 형태를 나타내고 있습니다. 비정상적인 세포들은 여러층이 증식하여...,"Specimen: Other(잇몸)<br>Appearance: soft, Size: 5X5X5cm<br>Operation: incisional<br>Recurrence: N, Lymph: N<br>",1
...,...,...,...,...,...
48648,"<p>Soft tissue sarcoma, grade II</p>","<p>의뢰된 좌측 후지 피부 종괴는 연부조직육종(soft tissue sarcoma, STS)으로 확인됩니다. STS는 피부에 발생하는 다양한 종류의 간엽 세포 유래 악성 종양을 통틀어서 지칭하는 용어로, 의...","<p>의뢰된 직경 3.5 cm 이상의 종괴의 1단면을 2개 블록에 나누어 제작-판독하였습니다. 결절은 기질 침습성(stromal invasion)을 나타내는 종양 세포의 증식 소견으로 관찰되며, 원형 또는 타...","<p>Specimen: Skin<br>Appearance: 단독 single, Size: 5X5X3cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>&nb...",1
48663,"<p>Osteosarcoma, Surface (juxtacortical) origin suspected</p>","<p>의뢰된 좌측 femur의 병변은 골육종(osteosarcoma, OSA)으로 진단되었습니다. OSA는 개의 뼈에서 발생하는 악성 종양의 85%를 차지하는 종양으로 평균 발생 연령은 7세 정도로 중년령 이...","<p>의뢰된 종괴의 2단면이 1차 제작되었고, 탈회(decalcification) 과정을 거쳐 longitudinal 1단면을 추가 제작하여 총 3단면 제작-판독하였습니다. 골표면에서 골질을 녹이며 성장하고 ...","<p>Specimen: Other(Lt. femur)<br>Appearance: 단독 single,산재 diffuse, Size: 6X3X2cm<br>Operation: excisional<br>Recurre...",1
48664,"<p>Osteosarcoma, Surface (juxtacortical) origin suspected</p>","<p>의뢰된 좌측 femur의 병변은 골육종(osteosarcoma, OSA)으로 진단되었습니다. OSA는 개의 뼈에서 발생하는 악성 종양의 85%를 차지하는 종양으로 평균 발생 연령은 7세 정도로 중년령 이...","<p>의뢰된 종괴의 2단면이 1차 제작되었고, 탈회(decalcification) 과정을 거쳐 longitudinal 1단면을 추가 제작하여 총 3단면 제작-판독하였습니다. 골표면에서 골질을 녹이며 성장하고 ...","<p>Specimen: Other(Lt. femur)<br>Appearance: 단독 single,산재 diffuse, Size: 6X3X2cm<br>Operation: excisional<br>Recurre...",1
48665,"<p>Osteosarcoma, Surface (juxtacortical) origin suspected</p>","<p>의뢰된 좌측 femur의 병변은 골육종(osteosarcoma, OSA)으로 진단되었습니다. OSA는 개의 뼈에서 발생하는 악성 종양의 85%를 차지하는 종양으로 평균 발생 연령은 7세 정도로 중년령 이...","<p>의뢰된 종괴의 2단면이 1차 제작되었고, 탈회(decalcification) 과정을 거쳐 longitudinal 1단면을 추가 제작하여 총 3단면 제작-판독하였습니다. 골표면에서 골질을 녹이며 성장하고 ...","<p>Specimen: Other(Lt. femur)<br>Appearance: 단독 single,산재 diffuse, Size: 6X3X2cm<br>Operation: excisional<br>Recurre...",1


## 5. 경계 사례 검토
점수가 동률이거나 둘 다 낮은 행을 확인하여 잘못된 분류를 보정합니다.
<sub>예: `diagnosis`에 adenoma, `micro`에 carcinoma가 언급된 혼합 표현을 사람이 빠르게 검토해 룰셋을 개선합니다.</sub>

In [8]:
uncertain = norm_df[norm_df['label'].isin(['uncertain', 'none'])]
uncertain[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma', 'score_adenocarcinoma']].head(10)


,DIAGNOSIS,COMMENTS,MICROSCOPIC_FINDINGS,GROSS_FINDINGS,score_adenoma,score_adenocarcinoma
0,"Mast cell tumor (Well-differentiated), completely excised","의뢰된 좌측 후지-옆구리 쪽 피부 종양은 비만세포종(mast cell tumor)으로 확인되었습니다. Mast cell tumor는 고양이 피부 종양의 약 21%를 차지하며, 분화 정도나 조직학적 분류와는 관...","의뢰된 조직의 장축면을 제작-판독하였습니다. 경계가 비교적 명확한 직경 약 4.3 mm의 mass는 원형 세포로 구성되어 있었으며, 종양세포는 잘 분화된 비만세포로 확인되었습니다. 종양 세포는 metachro...","Specimen: Skin<br>Appearance: 단독 single,soft,유동적 movable, Size: 0.5X0.5X0.5cm<br>Operation: excisional<br>Recurrence...",0,0
1,"Mast cell tumor (Well-differentiated), completely excised","의뢰된 좌측 후지-옆구리 쪽 피부 종양은 비만세포종(mast cell tumor)으로 확인되었습니다. Mast cell tumor는 고양이 피부 종양의 약 21%를 차지하며, 분화 정도나 조직학적 분류와는 관...","의뢰된 조직의 장축면을 제작-판독하였습니다. 경계가 비교적 명확한 직경 약 4.3 mm의 mass는 원형 세포로 구성되어 있었으며, 종양세포는 잘 분화된 비만세포로 확인되었습니다. 종양 세포는 metachro...","Specimen: Skin<br>Appearance: 단독 single,soft,유동적 movable, Size: 0.5X0.5X0.5cm<br>Operation: excisional<br>Recurrence...",0,0
4,"Mammary duct ectasia, mastitis","Mammary duct ectasia는 도관이 확장된 병변으로, 주로 중성화하지 않은 암컷에서 발정주기에 따라 성호르몬의 영향으로 발생하며, 유선 종양 주위에 형성되기도 합니다.<br>의뢰된 유선 검체에서는 ...","의뢰된 유선 검체 저배율 시야에서는 다수의 유선 도관 확장이 특징적인 소견으로 확인됩니다. 병변의 경계는 적절히 확보되었습니다. 확장된 유선 사이로는 mild하게 과증식한 유선 조직이 분포하고 있으며, 기질로...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.5X0.5X0.3cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...",1,1
5,"Mammary duct ectasia, mastitis","Mammary duct ectasia는 도관이 확장된 병변으로, 주로 중성화하지 않은 암컷에서 발정주기에 따라 성호르몬의 영향으로 발생하며, 유선 종양 주위에 형성되기도 합니다.<br>의뢰된 유선 검체에서는 ...","의뢰된 유선 검체 저배율 시야에서는 다수의 유선 도관 확장이 특징적인 소견으로 확인됩니다. 병변의 경계는 적절히 확보되었습니다. 확장된 유선 사이로는 mild하게 과증식한 유선 조직이 분포하고 있으며, 기질로...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.5X0.5X0.3cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...",1,1
6,"Mammary duct ectasia, mastitis","Mammary duct ectasia는 도관이 확장된 병변으로, 주로 중성화하지 않은 암컷에서 발정주기에 따라 성호르몬의 영향으로 발생하며, 유선 종양 주위에 형성되기도 합니다.<br>의뢰된 유선 검체에서는 ...","의뢰된 유선 검체 저배율 시야에서는 다수의 유선 도관 확장이 특징적인 소견으로 확인됩니다. 병변의 경계는 적절히 확보되었습니다. 확장된 유선 사이로는 mild하게 과증식한 유선 조직이 분포하고 있으며, 기질로...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.5X0.5X0.3cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...",1,1
7,"Mammary duct ectasia, mastitis","Mammary duct ectasia는 도관이 확장된 병변으로, 주로 중성화하지 않은 암컷에서 발정주기에 따라 성호르몬의 영향으로 발생하며, 유선 종양 주위에 형성되기도 합니다.<br>의뢰된 유선 검체에서는 ...","의뢰된 유선 검체 저배율 시야에서는 다수의 유선 도관 확장이 특징적인 소견으로 확인됩니다. 병변의 경계는 적절히 확보되었습니다. 확장된 유선 사이로는 mild하게 과증식한 유선 조직이 분포하고 있으며, 기질로...","Specimen: Mammary<br>Appearance: 다발 multiple, Size: 0.5X0.5X0.3cm<br>Operation: excisional<br>Recurrence: N, Lymph: ...",1,1
8,섬유종(Fibroma),섬유아세포에서 유래하는 양성종양입니다. 일부 세포에서 멜라닌 색소처럼 보이는 검은색소가 관찰되기도 하지만 전반적인 특징이 흑색종은 아닌 것으로 판단됩니다. 의뢰된 종괴의 경계부위가 명확히 다 관찰되지는 않으므...,"진피층에서 방추형의 종양세포 침윤이 관찰되는데, 종양세포는 상피층을 침습하지 않으며, 피막화로 구분되어 있지는 않으나 주위 결합조직과 어느 정도 구분되는 양상입니다. 종양세포는 길죽한 타원형 또는 방추형의 핵...","Specimen: Skin<br>Appearance: 단독 single, Size: 0.1X0.1X0.1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",0,0
9,섬유종(Fibroma),섬유아세포에서 유래하는 양성종양입니다. 일부 세포에서 멜라닌 색소처럼 보이는 검은색소가 관찰되기도 하지만 전반적인 특징이 흑색종은 아닌 것으로 판단됩니다. 의뢰된 종괴의 경계부위가 명확히 다 관찰되지는 않으므...,"진피층에서 방추형의 종양세포 침윤이 관찰되는데, 종양세포는 상피층을 침습하지 않으며, 피막화로 구분되어 있지는 않으나 주위 결합조직과 어느 정도 구분되는 양상입니다. 종양세포는 길죽한 타원형 또는 방추형의 핵...","Specimen: Skin<br>Appearance: 단독 single, Size: 0.1X0.1X0.1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",0,0
10,Fibroadnexal hamartoma,피부부속기 과오종(Fibroadnexal hamartoma)은 피부 부속기 및 결합조직이 증식하는 비종양성 병변입니다. 단순 증식성 병변이므로 외과적 제거를 통해 완치 가능합니다. 현재 종양성 변화나 두드러지...,"의뢰된 우측 전지 피부 검체 저배율 시야에서는 진피층 내로 피부 부속기와 콜라겐성 결합조직의 증식 소견이 확인됩니다. 증식한 결합조직 사이로 모낭과 땀샘 도관이 다소 증식했고, 땀샘 도관은 mild하게 확장되...","Specimen: Skin<br>Appearance: 단독 single, Size: 1X1X1cm<br>Operation: excisional<br>Recurrence: N, Lymph: N<br>",0,0
11,Fibroadnexal hamartoma,피부부속기 과오종(Fibroadnexal hamartoma)은 피부 부속기 및 결합조직이 증식하는 비종양성 병변입니다. 단순 증식성 병변이므로 외과적 제거를 통해 완치 가능합니다. 현재 종양성 변화나 두드러지...,"의뢰된 우측 전지 피부 검체 저배율 시야에서는 진피층 내로 피부 부속기와 콜라겐성 결합조직의 증식 소견이 확인됩니다. 증식한 결합조직 사이로 모낭과 땀샘 도관이 다소 증식했고, 땀샘 도관은 mild하게 확장되...","Specimen: S

## 6. 종(개/고양이) 판정
<sub>숫자 표현(예: `두 개`, `3 개의`)에 등장하는 한글 `개`를 개체로 오인하지 않도록, 종을 암시하는 문맥/품종/영문 표현이 있을 때만 점수를 부여합니다. 확신이 어려우면 `uncertain`으로 남깁니다.</sub>


In [ ]:
species_patterns = {
    'dog': [
        r"canin",
        r"dog(s)?",
        r"개에서",
        r"개\s*환자",
        r"견",
        r"강아지",
    ],
    'cat': [
        r"felin",
        r"cat(s)?",
        r"고양이에서",
        r"고양이\s*환자",
        r"냥이",
    ],
}

breed_keywords = {
    'dog': [
        'poodle', 'toy', 'miniature', 'retriever', 'bulldog', 'beagle', 'shihtzu', 'shih', 'tzu',
        'yorkshire', 'maltese', 'pom', 'pomeranian', 'spitz', 'jindo', 'shepherd', 'collie',
    ],
    'cat': [
        'persian', 'ragdoll', 'bengal', 'munchkin', 'siam', 'siamese', 'fold', 'norwegian',
        'russian', 'blue', 'sphynx', 'cat',
    ],
}

def normalize_species_text(row):
    text = ' '.join(
        str(row.get(col, ''))
        for col in ['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS']
        if isinstance(row.get(col, ''), str)
    ).lower().replace('<br>', ' ')
    text = re.sub(r"[^0-9a-z가-힣\s]+", " ", text)
    return re.sub(r"\s+", " ", text).strip()

def score_species(row):
    text = normalize_species_text(row)
    tokens = set(text.split())
    scores = {'dog': 0, 'cat': 0}

    for label, patterns in species_patterns.items():
        for pat in patterns:
            if re.search(pat, text):
                scores[label] += 2

    for label, keywords in breed_keywords.items():
        for kw in keywords:
            if kw in tokens:
                scores[label] += 1

    if scores['dog'] == 0 and scores['cat'] == 0:
        species_label = 'uncertain'
    elif scores['dog'] > scores['cat']:
        species_label = 'dog'
    elif scores['cat'] > scores['dog']:
        species_label = 'cat'
    else:
        species_label = 'uncertain'

    return pd.Series({
        'score_dog': scores['dog'],
        'score_cat': scores['cat'],
        'species_label': species_label,
    })

species_df = norm_df.apply(score_species, axis=1)
norm_df = pd.concat([norm_df, species_df], axis=1)

norm_df[['DIAGNOSIS', 'species_label', 'score_dog', 'score_cat']].head()


## 7. 양성/악성 판정
<sub>악성과 양성 모두에서 흔히 쓰이는 조직명 표현을 규칙으로 점수화하고, 근거가 약하면 `uncertain`으로 남깁니다.</sub>

In [ ]:
benign_patterns = [
    r"adenoma",
    r"papilloma",
    r"polyp",
    r"fibroma",
    r"lipoma",
    r"osteoma",
    r"chondroma",
    r"hemangioma",
    r"lymphangioma",
    r"leiomyoma",
    r"rhabdomyoma",
    r"neuroma",
    r"fibroadenoma",
    r"mixed tumor",
    r"melanocytoma",
    r"histiocytoma",
]

malignant_patterns = [
    r"carcinoma",
    r"adenocarcinoma",
    r"sarcoma",
    r"fibrosarcoma",
    r"osteosarcoma",
    r"hemangiosarcoma",
    r"liposarcoma",
    r"leiomyosarcoma",
    r"rhabdomyosarcoma",
    r"carcinosarcoma",
    r"malignant mixed tumor",
    r"melanoma",
    r"lymphoma",
    r"mast cell tumor",
    r"histiocytic sarcoma",
]

behavior_support = {
    'benign': ['benign', '양성'],
    'malignant': ['malignant', 'invasive', 'metastatic', 'metastasis', 'anaplastic', '악성'],
}

def concat_norm_text(row):
    return ' '.join(str(row.get(f"{col}_norm", '')) for col in norm_cols)

def score_behavior(row):
    text = concat_norm_text(row)
    scores = {'benign': 0, 'malignant': 0}

    for pat in benign_patterns:
        if re.search(pat, text):
            scores['benign'] += 2
    for pat in malignant_patterns:
        if re.search(pat, text):
            scores['malignant'] += 2

    for label, keywords in behavior_support.items():
        for kw in keywords:
            if kw in text:
                scores[label] += 1

    if scores['benign'] == 0 and scores['malignant'] == 0:
        behavior_label = 'none'
    elif scores['benign'] > scores['malignant']:
        behavior_label = 'benign'
    elif scores['malignant'] > scores['benign']:
        behavior_label = 'malignant'
    else:
        behavior_label = 'uncertain'

    return pd.Series({
        'score_benign': scores['benign'],
        'score_malignant': scores['malignant'],
        'behavior_label': behavior_label,
    })

behavior_df = norm_df.apply(score_behavior, axis=1)
norm_df = pd.concat([norm_df, behavior_df], axis=1)

norm_df[['DIAGNOSIS', 'behavior_label', 'score_benign', 'score_malignant']].head()


## 8. 확실 사례 스냅샷
<sub>두 진단군 각각에서 패턴이 제대로 작동하는지, 해부 부위나 진단 요약이 기대대로 나오는지 빠르게 눈으로 검증합니다.</sub>

In [ ]:
# adenoma 확실 사례 스냅샷
adenoma_rows = norm_df[norm_df['label'] == 'mammary_adenoma']
adenoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenoma', 'behavior_label', 'species_label']].head(10)


In [ ]:
# adenocarcinoma 확실 사례 스냅샷
adenocarcinoma_rows = norm_df[norm_df['label'] == 'mammary_adenocarcinoma']
adenocarcinoma_rows[['DIAGNOSIS', 'COMMENTS', 'MICROSCOPIC_FINDINGS', 'GROSS_FINDINGS', 'score_adenocarcinoma', 'behavior_label', 'species_label']].head(10)


## 9. 요약 통계
<sub>라벨 분포와 점수 분포를 보면 규칙 조정이 필요한지 판단할 수 있습니다.</sub>

In [11]:
total = len(norm_df)
label_counts = norm_df['label'].value_counts()
behavior_counts = norm_df['behavior_label'].value_counts()
species_counts = norm_df['species_label'].value_counts()

print(f"Total rows: {total}")
print("
Mammary label counts:
", label_counts)
print("
Behavior label counts:
", behavior_counts)
print("
Species label counts:
", species_counts)

score_summary = norm_df[['score_adenoma', 'score_adenocarcinoma', 'score_benign', 'score_malignant', 'score_dog', 'score_cat']].describe()
score_summary


Total rows: 48692
label
none                      31659
mammary_adenoma            8666
mammary_adenocarcinoma     5225
uncertain                  3142
Name: count, dtype: int64


,score_adenoma,score_adenocarcinoma
count,48692.000000,48692.000000
mean,0.535098,0.445330
std,1.060889,0.998525
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,1.000000
max,6.000000,6.000000


## 10. 정제된 데이터셋 저장
<sub>Adenoma/Adenocarcinoma 라벨과 점수를 포함한 CSV를 남겨 후속 분석·모델 학습에서 동일한 결과를 재사용할 수 있습니다.</sub>

In [12]:
output_cols = [
    'INSP_RQST_NO', 'FOLDER', 'FILE_NAME', 'SITE',
    'DIAGNOSIS', 'GROSS_FINDINGS', 'COMMENTS', 'MICROSCOPIC_FINDINGS',
    'score_adenoma', 'score_adenocarcinoma', 'score_benign', 'score_malignant',
    'score_dog', 'score_cat', 'label', 'behavior_label', 'species_label'
]
curated = norm_df[output_cols].copy()
base_path = Path('./mammary_adenoma_vs_adenocarcinoma_curated')
parquet_path = base_path.with_suffix('.parquet')
csv_path = base_path.with_suffix('.csv')
curated.to_parquet(parquet_path, index=False)
curated.to_csv(csv_path, index=False)
parquet_path, csv_path


(PosixPath('mammary_adenoma_vs_adenocarcinoma_curated.parquet'),
 PosixPath('mammary_adenoma_vs_adenocarcinoma_curated.csv'))

## 11. 다음 단계
- 경계 사례 수동 검토 후 keyword 리스트를 보완합니다.
- 희귀 동의어에 대한 recall을 높이려면 TF-IDF나 임베딩 유사도 방식을 고려합니다.
<sub>규칙→수동 리뷰→키워드 보완→모델 전환의 반복 학습 루프를 염두에 둔 로드맵입니다.</sub>